---
title: "Deep Learning - Notes"
author: Kalle Bylin
date: 2021-03-01
categories: [deep-learning]
toc: true
format:
  html:
    code-fold: true
jupyter: python3
---


> A short summary of DL fundamentals.

# About

This notebook contains my notes for introductory DL concepts.

## What is a perceptron?

- Perceptrons were originally brain models created to understand how the brain works. A perceptron as we know it encodes several principles about how the brain works and then evolved into an algorithm for supervised binary classification.

In the 1960's, Frank Rosenblatt published the book `Principles of Neurodynamics: Perceptrons and the Theory of Brain Mechanisms`. It is curious that this fundamental block for AI was, in the author's mind, a tool for understanding the human brain and not for pattern recognition (even though he encouraged this use as well):

> For this writer, the perceptron program is _not_ primarily concerned with the invention of devices for "artificial intelligence", but rather with investigating the physical structures and neurodynamic principles which underlie "natural intelligence". *A perceptron is first and foremost a brain model, not an invention for pattern recognition* [emphasis added]" (p. viii). 

In other words, the perceptron is actually a simplification and abstraction which has allowed us to discover principles for how the brain works. These same principles were then also used to create pattern recognition machines.

Rosenblatt explicitly recognizes that his model is a direct descendant of the model created by McCulloch and Pitts, and influenced by the theories of Hebb and Hayek. 

Main components of a perceptron in Rosenblatt's book:
- **Environment**: The environment generates the information that is initially passed on to the perceptron.


- **Signal generating units**: Each unit receives a signal and generates an output signal.


- **Signal propagation functions**: These are rules that define how signals are generated and transmitted.


- **Memory functions**: These are rules that define how properties of the perceptron can be changed in response to certain activity.

The definition of a single neuron evolves from the ideas above.

A neuron takes values from its environment (e.g. x1, x2, x3) and each of these gets multiplied by a stored parameter (e.g. w1, w2, w3). The sum of each of these operations is then passed through an activation function.

In other words, it's as if we are trying to pass a signal through the neuron and all of these components work together to establish how the signal is transmitted. 


We can imagine three old batteries used to turn on a machine. Turning it on with too little energy could cause it to break, so we check the current before passing it on to the machine. 

![Perceptron](img/perceptron_batteries.png)

In the example below, we are randomly initializing the parameters. The activation function is a step-function with a threshold of 4. This means that the signal is only passed on as a unitary value if it is larger than or equal to the threshold.

In [1]:
import numpy as np

w1 = np.random.random()*2 # generates random floats between 0 and 2
w2 = np.random.random()*2
w3 = np.random.random()*2 

print(f'W1 is equal to: {w1}')
print(f'W2 is equal to: {w2}')
print(f'W3 is equal to: {w3}\n')

x1 = 2
x2 = 1
x3 = 3
b = 0

activation_function = lambda x: 1 if x >=4 else 0

output = activation_function(x1*w1 + x2*w2 + x3*w3)

print('Output:', output)

if output == 1:
    print('The machine is on!')
else:
    print('Not enough energy to turn on the machine :(')

W1 is equal to: 0.6321002815675523
W2 is equal to: 1.6995522179113192
W3 is equal to: 1.2072466687862997

Output: 1
The machine is on!


## What is an activation function?

Going back to Rosenblatt's book, activation functions are essentially signal propagation functions. 

When a neuron receives a signal, the activation function decides if the signal is passed on and how strong the output signal becomes. 

We already learned about the step function. This activation is often not ideal for multiple reasons.

First of all, the result is binary. But sometimes we are more interested in also knowing the degree of certainty, so I probability might be better.

We might also want to have a wider range of values. When predicting age, for example, binary values of 0 or 1 will be of little value. 

A lot of different activation functions have been developed by researchers. Three common activation functions are:

**Sigmoid function**
Has a great property of having outputs between 0 and 1 and therefore can be interpreted as probabilities. The function is also smooth and easy to differentiate which makes learning easier.

It can be problematic when the input signal has very large positive or negative values. At those points the derivative is very close to 0 and learning becomes very slow.

**Hyperbolic function**
This is another smooth function with a range of values between -1 and 1. This is interesting because sometimes a signal might have a reverse effect on the output and the hyperbolic function allows us to include this type of relationship in the network. 

**Rectified linear unit (ReLU)**
This function is very simple aned fast to compute. This allows us to work with larger and more complex models that, given enough data, can produce better results overall.

## What is a neural network?

- A neural network is a directed system of connected neurons which is capable of propagating a signal received from the environment and produces an output signal.

The neurons are generally organized in different layers which allow us to break free from linearity. 

Linearity means that we are also assuming monotonicity. In other words, an increase in an input value always increases the output value if the corresponding weight is positive (and will always decrease the output value if the weight is negative).

This works fine in examples like the one seen above where we turn on a lightbulb depending on how much energy we get from each battery.

But this makes less sense in other cases. For examples very pronounced sales spikes in an online marketplace followed by no sales at all could be indicative of fraud. A positive value for a feature describing this type of spikes should increase the probability of fraud.

But is this always the case?

What if we additionally know that the owner of the store is a web celeb (网红) on Alibaba? In that case the spikes above are expected, and therefore a positive value for this feature should decrease the probability of fraud while any other behavior might be suspicious.


**Let's build a neural network step by step:**

The simplest example is a single neuron with only one parameter which does not modify the signal in any way:

In [2]:
def nn(x, w):
    return w*x

print(nn(3, 6))

18


Most processes we find in the real world are not this simple. We need to take into account multiple input features. 

I love music, imagine I want to predict if I will like a particular song or not. We can quickly multiply each input value by its corresponding weight using a dot product.

In [3]:
def nn(X, W):
    return W.dot(X)

x1 = 3.5 # length of songs in minutes
x2 = 0 # binary value indicating if the genre is jazz or not
x3 = 0 # binary value indicating if the artist is Nicki Minaj or not
x4 = 1 # binary value indicating if the artist is Alan Walker

X = np.array([x1, x2, x3, x4])
W = np.array([1., 0.3, -3, 4])

print(nn(X, W))

7.5


What is interesting above is that the weights actually encode certain information about my taste in music. I really don't like when a song is too short and, in general, I'm not fond of music by Nicki Minaj (notice the negative weight). But I do love music by Alan Walker. Jazz is nice but I don't love it. 

Now the real question is, how do we know the values of these weights. In this case, I used my deep expertise regarding my own music taste to set the weights. But we would usually be more interested in doing this for the users of our streaming platform at scale. Millions of people we have never and will probably never meet in person. 

So, first we're going to change the output. We want to predict if a particular user is going to like a song or not. Using the sigmoid function we can directly compare the true values (0 or 1) to our predictions (floats between 0 and 1).

We will also initialize the weights randomly and print out an error value showing how close our prediction is to the true value:

In [4]:

def nn(X, y, W): 
    prediction = W.dot(X)
    error = y-prediction
    
    print('Predicted value:', round(prediction, 3))
    print('Error:', round(error, 3))

x1 = 3.5 # length of songs in minutes
x2 = 0 # binary value indicating if the genre is jazz or not
x3 = 0 # binary value indicating if the artist is Nicki Minaj or not
x4 = 1 # binary value indicating if the artist is Alan Walker

X = np.array([x1, x2, x3, x4])
y = 5
W = np.random.random(size=4)*2-1

nn(X, y, W)

Predicted value: -1.468
Error: 6.468


It's now time to start talking about learning or how our neural network is going to choose the best parameters during training to make accurate predictions.

We will follow a very simple framework of three steps:

1. Predict
2. Compare 
3. Learn

This time we will add multiple iterations to our algorithm and we will do the three steps above during each iteration.



In [5]:
def nn(X, y, W): 
    print(f'True value: {y}\n')
    
    for i in range(10):
        prediction = W.dot(X)
        error = y-prediction

        print('Predicted value:', round(prediction, 3))
        print('Error:', round(error, 3))
        
        if error > 0:
            W += 0.01
        else:
            W -= 0.01  

x1 = 3.5 # length of songs in minutes
x2 = 0 # binary value indicating if the genre is jazz or not
x3 = 0 # binary value indicating if the artist is Nicki Minaj or not
x4 = 1 # binary value indicating if the artist is Alan Walker

X = np.array([x1, x2, x3, x4])
y = 1
W = np.random.random(size=4)*2-1

nn(X, y, W)


True value: 1

Predicted value: 0.116
Error: 0.884
Predicted value: 0.161
Error: 0.839
Predicted value: 0.206
Error: 0.794
Predicted value: 0.251
Error: 0.749
Predicted value: 0.296
Error: 0.704
Predicted value: 0.341
Error: 0.659
Predicted value: 0.386
Error: 0.614
Predicted value: 0.431
Error: 0.569
Predicted value: 0.476
Error: 0.524
Predicted value: 0.521
Error: 0.479


Here we are doing hot & cold learning. After each prediction we are making a comparison just like in the classical game so that we get "hotter" or closer to the true answer after each guess.

The way we are calculating the error makes it negative if it was too high or positive if our guess was too low.

We can now be a bit smarter in the way we learn. Instead of guessing and trying to jiggle the output to both sides, we can use the gradient to guide us:

In [6]:
def nn(X, y, W): 
    print(f'True values: {y}\n')
    n = y.shape[1]
    
    for i in range(100):
        predictions = W.dot(X.T)
        mse = (1/2)*np.mean((y-predictions)**2)
        
        if i % 10 == 0:
            print('Mean Squared Error:', round(mse, 3))
        
        dW = -(1/n)*(y-predictions).dot(X)
        W -= 0.1*dW
          
    print('\nW:', np.round(W, 3))
    print('\nFinal predictions:', np.round(predictions, 2))

X = np.array([[1, 0],
              [0, 1],
              [0, 0],
              [1, 1]])

y = np.array([[1, 0, 0, 1]])
W = np.random.randn(1, 2)

nn(X, y, W)

True values: [[1 0 0 1]]

Mean Squared Error: 0.234
Mean Squared Error: 0.132
Mean Squared Error: 0.078
Mean Squared Error: 0.046
Mean Squared Error: 0.028
Mean Squared Error: 0.017
Mean Squared Error: 0.01
Mean Squared Error: 0.006
Mean Squared Error: 0.004
Mean Squared Error: 0.002

W: [[0.927 0.073]]

Final predictions: [[0.92 0.07 0.   1.  ]]


This version led us quickly very close to the correct answers. But we now have another problem. The previous problem was very easy to learn because one of the features had a direct 1-on-1 relationship with an output. A simple linear function was capable of leveraging that feature.

Let's look at the next example:

In [7]:
X = np.array([[1, 0],
              [0, 1],
              [0, 0],
              [1, 1]])

y = np.array([[1, 1, 0, 0]])
W = np.random.randn(1, 2)

nn(X, y, W)

True values: [[1 1 0 0]]

Mean Squared Error: 0.863
Mean Squared Error: 0.572
Mean Squared Error: 0.408
Mean Squared Error: 0.311
Mean Squared Error: 0.254
Mean Squared Error: 0.219
Mean Squared Error: 0.198
Mean Squared Error: 0.186
Mean Squared Error: 0.178
Mean Squared Error: 0.174

W: [[0.462 0.204]]

Final predictions: [[0.47 0.2  0.   0.67]]


MSE might not be extremely high but the answers are terrible. This is because the new problem we're working with cannot be solved with a simple linear function. So, to solve this we can add a hidden layer with an activation function (more on why this works can be found further down): 

In [18]:
np.random.seed(2)

def relu(x):
    return (x>0)*x

def relu_prime(x):
    return x>0

def nn(X, y, hidden_size , lr=0.01): 
    print(f'True values: {y}\n')
    n = y.shape[1]
    
    W1 = np.random.randn(X.shape[1], hidden_size)
    W2 = np.random.randn(hidden_size, 1)
    
    for i in range(100):
        z1 = X.dot(W1)
        a1 = relu(z1)
        
        z2 = a1.dot(W2) # predictions
        mse = (1/2)*np.mean((y-z2)**2)
        
        if i % 10 == 0:
            print('Mean Squared Error:', round(mse, 3))
        
        dZ2 = (y-z2)
        dW2 = a1.T.dot(dZ2)
        W2 += lr*dW2
        
        dZ1 = (dZ2.dot(W2.T))*relu_prime(a1)
        dW1 = X.T.dot(dZ1)
        W1 += lr*dW1 #(1/n)
    
    print('\nW1:', np.round(W1, 3))
    print('W2:', np.round(W2, 3))
    
    print('\nFinal predictions:', np.round(z2, 2))

X = np.array([[1, 0],
              [0, 1],
              [0, 0],
              [1, 1]])

y = np.array([[1, 1, 0, 0]]).T

nn(X, y, 4)


True values: [[1]
 [1]
 [0]
 [0]]

Mean Squared Error: 1.12
Mean Squared Error: 0.284
Mean Squared Error: 0.133
Mean Squared Error: 0.078
Mean Squared Error: 0.052
Mean Squared Error: 0.037
Mean Squared Error: 0.027
Mean Squared Error: 0.021
Mean Squared Error: 0.016
Mean Squared Error: 0.012

W1: [[-0.417 -0.056 -2.136  0.599]
 [-1.793 -0.842  0.847 -1.302]]
W2: [[-1.058]
 [-0.909]
 [ 0.876]
 [ 1.738]]

Final predictions: [[1.04]
 [0.74]
 [0.  ]
 [0.  ]]


## What is universality?

- Universality is the ability to compute any arbitrary function.

It has been proven that any function we can think of can be approximated by a neural network with at least two layers.

[Michael Nielsen clarifies](http://neuralnetworksanddeeplearning.com/chap4.html#two_caveats) two caveats related to this idea of universality:
1. **Universality doesn't mean that the neural network is guaranteed to be exact**, instead we are approximating the function and the more neurons we add the closer it gets to the true function. This means that for any function *f(X)* and any error threshold we define we can always find a neural network with output *g(x)* that satisfies: 

$$ |g(x) - f(x)| < \epsilon $$


2. **Neural networks approximate continuous functions**. Functions that are not continuous with sharp and sudden jumps won't be approximated by a neural network as a general rule. But that doesn't mean that we can often use a continuous approximation that is good enough for our given purposes when trying to approximate a discontinuous function.